Credit Risk Prediction ML Project!
Serves as a way to see whether or not a person's loan application will default based on credit. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

